# Nice finds
* http://dbpedia.org/page/WTRF-TV

# We need
* topic (e.g. JEX domain classifier)
* local news sections of websites

# TODO
* fix name values in json

In [1]:
import json
import operator
from SPARQLWrapper import SPARQLWrapper, JSON
from collections import defaultdict

In [2]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

In [3]:
def find_dbpedia_of_publisher(publisher_string):
    """
    
    """
    sparql.setQuery("""
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        SELECT ?page
        WHERE { ?page rdfs:label "%s"@en }
    """ % publisher_string)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    dbpedia_links = set()
    for result in results["results"]["bindings"]:
        a_link = result['page']['value'] 
        if all([not a_link.startswith('http://dbpedia.org/resource/Category'),
                a_link.startswith('http://dbpedia.org/resource/')]):
            dbpedia_links.add(a_link)
    return dbpedia_links

## load publishers

In [4]:
use_cache = False
if use_cache:
    with open('publishers_with_freq.json') as infile:
        publishers_with_freq = json.load(infile)
    
else:
    publishers_with_freq = defaultdict(int)
    with open('signalmedia-1m.jsonl') as infile:
        for line in infile:
            article = json.loads(line)
            publishers_with_freq[article['source']] += 1
    with open('publishers_with_freq.json', 'w') as outfile:
        json.dump(publishers_with_freq, outfile)

In [5]:
with open('locations.json') as infile:
    locations = json.load(infile)

print()
print('already done: %s' % (len(locations)-1))
for source, freq in sorted(publishers_with_freq.items(),
                           key=operator.itemgetter(1),
                           reverse=True):
    
    if source in locations:
        continue 
        
    dbpedia_links = find_dbpedia_of_publisher(source)
    print(source, freq)
    for dbpedia_link in dbpedia_links:
        print(dbpedia_link)
    print()
    homepage = input('homepage: ')
    dbpedia = input('dbpedia: ')
    wiki_uri = input('wikipedia: ')
    dbpedia_loc = input('dbpedia location: ')

    locations[source] = {'name': source}
    
    for key, value in [('location_dbpedia_uri', dbpedia_loc),
                       ('dbpedia_uri', dbpedia),
                       ('wiki_uri', wiki_uri),
                       ('homepage', homepage)]:
        if value:
            locations[source][key] = value
    
    with open('locations.json', 'w') as outfile:
        json.dump(locations, outfile)
    
    break
    
    



already done: 100
Virtual Strategy Magazine 931 set()

homepage: http://virtual-strategy.com/
dbpedia: 
wikipedia: 
dbpedia location: 
